In [1]:
import os
import sys
import json

sys.path.insert(0, '/mnt/d/projects/bassline_transcription')

from utilities import *

import pandas as pd
import numpy as np

In [2]:
directories = get_directories('../data/directories.json')
scales, track_dicts, track_titles = read_metadata(directories)

In [26]:
M = 8

minor_matrix, major_matrix = [], []
minor_titles, major_titles = [], []
for title in track_titles:
    
    try:
        vector = load_symbolic_representation(title, directories, M)
        scale_type = track_dicts[title]['Key'].split(' ')[-1]
        
        if scale_type == 'min':
            minor_matrix.append(vector)
            minor_titles.append(title)
            
        elif scale_type == 'maj':
            major_matrix.append(vector)
            major_titles.append(title)
            
    except:
        print('Error on {}'.format(title))
        
major_matrix = np.stack(major_matrix, axis=0)
minor_matrix = np.stack(minor_matrix, axis=0)

df_minor = pd.DataFrame(minor_matrix)
df_major = pd.DataFrame(major_matrix)

df_minor['Title'] = minor_titles
df_major['Title'] = major_titles

df_minor = df_minor.reindex(columns=['Title'] + [x for x in np.arange(len(vector))])
df_major = df_major.reindex(columns=['Title'] + [x for x in np.arange(len(vector))])

representation_path = os.path.join('..','data','datasets')

min_title = 'bassline_representations_min_M{}.csv'.format(M)
min_dir = os.path.join(representation_path, min_title)
df_minor.to_csv(min_dir, index=False, header=False)

maj_title = 'bassline_representations_maj_M{}.csv'.format(M)
maj_dir = os.path.join(representation_path, maj_title)
df_major.to_csv(maj_dir, index=False, header=False)

Error on Benihana – Quiero (Original Mix)
Error on CamelPhat & Elderbrook ‘Cola’ (Franky Rizardo Remix)
Error on Illyus & Barrientos ‘Scream’ (Extended Mix)
Error on Mark Knight & Funkagenda - Man With The Red Face - Radio Edit
Error on Michael Bibi - Dance To The Devil
Error on Rudimental & The Martinez Brothers – Easy On Me (Visualiser)


In [ ]:
min_count = 0
maj_count = 0
for track in track_dicts.values():
    
    key, scale_type = track['Key'].split(' ')
    
    if scale_type == 'min':
        min_count += 1
        
    if scale_type == 'maj':
        maj_count += 1    

## Creating The Representation

In [ ]:
#from transcription.representation import (transpose_to_C, fill_midi_array,
#                                          convert_to_vector,get_symbolic_representation)

In [ ]:
def load_numpy_midi(file_name, directory):  
    
    return np.load(os.path.join(directory, file_name))

def create_scale_pitches(key, typ):
    
    scale_pitches = scales[key][typ]['notes']

    midi_notes = [24 + pitches.index(pitch) for pitch in scale_pitches]
    midi_notes.extend([36 + pitches.index(pitch) for pitch in scale_pitches])
    
    return midi_notes

#M = 8

#midi_dir = os.path.join(directories['transcription']['midi_notes'], 'M={}'.format(M))
midi_dir = directories['transcription']['midi_notes']

file_names = next(os.walk(midi_dir))[-1]

In [ ]:
pitches = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

In [ ]:
N_bars = 4
frame_factor = 8
M=8

for file_name in file_names:
    
    midi_array = load_numpy_midi(file_name, midi_dir)
    
    title = file_name.split('.npy')[0]
    key, typ = track_dicts[title]['Key'].split(' ')
       
    N_intervals = pitches.index(key)
    
    
    if midi_array.shape[0] > 2:

        midi_array_T = transpose_to_C(midi_array, N_intervals)

        symbolic_representation = get_symbolic_representation(midi_array, N_bars, frame_factor, M)

        #print(len(symbolic_representation))
        
        print(symbolic_representation)

In [ ]:
indices = []
rows = []
for idx, midi in enumerate(midi_array[:-1]):
    
    end = midi[0] + midi[3]
    if end != midi_array[idx+1, 0]:
        indices.append(idx+1)
        rows.append([end , 0, 120, midi_array[idx+1, 0]-end])

midi_array = np.insert(midi_array, indices, rows, axis=0)

if midi_array[0,0] != 0.0:
    midi_array=np.insert(midi_array, 0, [0.0, 0, midi_array[0,2], midi_array[0,0]], axis=0)

end = midi_array[-1,0] + midi_array[-1,3]
if end != N_bars*4:
    midi_array=np.insert(midi_array, -1, [end, 0, midi_array[-1,2], N_bars*4-end], axis=0)

In [ ]:
delta = 0.25 * (frame_factor / M) # N_pqb

vector = []
for midi in midi_array:
    
    note = midi[1]
    duration = (midi[3]/delta).astype(int)
    
    vector += [note]*duration

## Analysis

In [ ]:
minor_pitches, major_pitches = [], []
minor_pitches_T, major_pitches_T = [], []

for file_name in file_names:
    
    midi_array = load_numpy_midi(file_name, midi_dir)
    
    title = file_name.split('.npy')[0]
    key, typ = track_dicts[title]['Key'].split(' ')
       
    N_intervals = pitches.index(key)
    
    if midi_array.size > 1:
        
        notes = midi_array[:,1].astype(int)
        
        #transpose
        notes_T = [note-N_intervals for note in notes]
                
        midi_array[:,1] = notes_T        
        
        if typ == 'maj':
            major_pitches.extend(notes)
            major_pitches_T.extend(notes_T)
        else:
            minor_pitches.extend(notes)
            minor_pitches_T.extend(notes_T)
            
print('Lowest Major Key Midi Idx: {}\nAfter Transposition: {}'.format(min(major_pitches),min(major_pitches_T)))
print('\nLowest Minor Key Midi Idx: {}\nAfter Transposition: {}'.format(min(minor_pitches),min(minor_pitches_T)))

## Sketcehs

if key_id <= 5:
    notes_T = [note-key_id for note in notes]
else:
    notes_T = [note+(12-key_id) for note in notes]
    
    
        if min(notes_T) < 24:
            print('Key: {} {}, N_intervals: {}'.format(key,typ,N_intervals))
            print('Scale MIDI: {}'.format(create_scale_pitches(key, typ)))
            print('Pitches: {}\nTransposed: {}\n'.format(notes, notes_T))

idx = 3

2*(pitches[idx-12:] + pitches[:idx])